[Return to README](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/README.md#explore-the-project)

# Wikipedia Notable Life Expectancy

## Notebook 1:  Data Collection

## Context

This case study was chosen by this author as an end-to-end portfolio project for the following reasons:  
1. accessibility of the data,
2. potential for use and demonstration of a wide range of skills, including coding in Python (using Pycharm and Jupyter Notebooks), version control (using Git, GitHub, and ReviewNB), web scraping (using Scrapy), relational database management (using SQLite and [SQLite Viewer](https://inloop.github.io/sqlite-viewer/)),  data cleaning (built-in string methods, regular expressions, and pandas), Exploratory Data Analysis (EDA using numpy, pandas, matplotlib, plotly, and seaborn), data preprocessing, unsupervised learning (Kmeans and hierachical clustering), supervised learning (regression models), and user interface for predictions,
3. in particular, its potential for regression modeling to gain more exposure to its associated algorithms, as this author had relative broader experience with classification models in past projects.
    

## Objective

The...

## Data Dictionary
Variable: Description

<a id=read a></a>
The [Web-scraping step](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_collect_thanak_2022_06_10.ipynb) is reproducible, with the following caveats:
1. Wikipedia evolves with contribution and editing, so scraping even the identical dates will likely result in at least some minor variation in the raw dataset.
2. Converting the raw data for what an individual was known, into a predictive categorical feature, is a programmatically-driven manual process.  Though faster than manual entry, and arguably more accurate, it has the same specificity to the dataset.  If new data is scraped, particularly for a different time period, it will need this cleaning step recoded to match the new data (no small feat!).
3. The [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py) file is reused for all [spiders](https://github.com/teresahanak/wikipedia-life-expectancy/tree/main/wikipedia_life_expectancy/spiders) within the [Scrapy project folder](https://github.com/teresahanak/wikipedia-life-expectancy/tree/main/wikipedia_life_expectancy), so requires updating to match the spider that will be crawled.  [Frank Andrade's course](https://www.udemy.com/share/1050RC3@hQ2ccLlpcvivy0WcwxoyY-pTDZW1ATRujXa3_csxIPkk6f4MV3Ogh20gLzy_BSAf/) instructions for Scrapy were followed.
4. Due to 1 and 2, reproducing the scraping steps merely as an exercise in Web scraping with Scrapy is recommended, versus aiming to duplicate the results here from start to finish.  For comparitive results, choosing any entry point after scraping (i.e., starting with the [complete raw dataset](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_raw_complete.db) in [Notebook 2](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean1_thanak_2022_06_13.ipynb), or any later version and its corresponding notebook) is the better option.

<a id=instruct a></a>
Follow [these instructions](https://www.youtube.com/watch?v=ooNngLWhTC4) from Frank Andrade, to set up Anaconda virtual environment and Scrapy.
1. Create virtual environment with Anaconda Navigator
2. Install Scrapy in the virtual environment
3. Synch PyCharm to the Anaconda virtual environment
4. Install protego



## Data Collection
- Data was collected from 6/9/22 to 6/10/22, using Scrapy. 

### 6/9/2022

- The [Wikipedia](https://en.wikipedia.org/wiki/Main_Page) page, [List of Deaths by Year](https://en.wikipedia.org/wiki/Lists_of_deaths_by_year), contains entries for as early as 1987, to the present day.  
- 1994 was chosen as the start year for collection as it is the first year with entries following the current format: "Name, age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, cause of death (if known), and reference."
- For ease of pagination, [Deaths in January 1994](https://en.wikipedia.org/wiki/Deaths_in_January_1994) was the start url for scraping, proceeding month by month through subsequent pages.
- Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) scraped `month_year`, `day`, `name`, `info` ("age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, and cause of death (if known)"), and `link` for each entry on each month's page.  
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table *wp_deaths_94_to_22* within [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  This scraping was successful for January, 1994 through May, 2022 data.
- [Deaths in 2022 -- June](https://en.wikipedia.org/wiki/Deaths_in_2022#June)--the current month's page, at the time of scraping--varied in format, and was therefore scraped separately, after `num_references` was scraped for the previous entries.
- The original order of entries was preserved by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) in [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  Scrapy trades pagination order for speed, which is noticable when pagination is of higher magnitude.  Therefore, scraping each entry's page for number of references was done separately, as the order was sure to vary. Spider ["references"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/references.py) scraped for number of references.  The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table *wp_reference_counts* in [wp_reference_counts.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_reference_counts_2.db), with the results falling ~13,000 rows short of the 133,769 rows captured by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py).
- Finally, the [June, 2022](https://en.wikipedia.org/wiki/Deaths_in_2022#June) page was scraped by Spider ["June_2022"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/June_2022.py), successfully capturing all of the previous fields, including number of references.  Note that the number of pages was some order of magnitude smaller.
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table *wp_deaths_June_2022* in [wp_deaths_June_2022.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_June_2022.db), resulting in 145 rows from the first part of June, 2022.

### 6/10/2022
The remaining data collection steps are outlined in this notebook:
1. [Reading, Sampling, and Checking Data Shape](#step1)
    - SQLite tables *wp_deaths_94_to_22*, *wp_reference_counts*, and *wp_June_2022* were read in as pandas dataframes.
2. [Combining Dataframes](#step2)
    - Dataframes for wp_deaths_94_to_22 and wp_reference_counts were combined using `link` as the unique identifier.
    - Dataframe for wp_deaths_June_2022 was added.
3. [Duplicate Rows](#step3)
    - 9 rows of duplicate entries were dropped.
4. [Missing Values](#step4)
    - 5 rows missing essential data were dropped.
    - 6 entries had missing `name` and `num_references`, but contained the name in the `info` feature, to be extracted later during data cleaning.  As these entries had no associated page, their `num_references` was set equal to to 0.
5. [Missing `num_references` Values](#step5)
    - A single modification was made to the original XPath for the original "by_year" Spider, to match a variation on the pages for links with missing `num_references`.  
    - Those pages were then were rescraped iteratively by Spiders ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py), ["refs3"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs3.py), ["refs4"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs4.py), and ["refs5"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs5.py).
    - The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote their respective results to SQLite tables *refs2* in [refs2.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs2.db), *refs3* in [refs3.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs3.db), *refs4* in [refs4.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs4.db), and *refs5* in [refs5.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs5.db).
    - During each iteration, the additional `num_references` were added to the main dataset and remaining links with missing `num_references` values were identified.
    - After the final iteration, 92 entries remained with missing `num_references` that were all missing an associated page, so `num_references` was set equal to to 0.  These entries contained all of the other relevant features, so were preserved.
    - The resultant raw dataset has 133,900 rows and 6 columns.

\*  The current version of [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py) reflects its use by the most recent project spider crawled, as it is reused for all spiders within the [Scrapy project folder](https://github.com/teresahanak/wikipedia-notable-deaths/tree/main/wikipedia_notable_deaths).

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

<a id='step1'></a>
### Reading, Sampling, and Checking Data Shape

### January 1994 through May 2022 Data (without reference counts)

In [2]:
# Reading the wp_deaths_94_to_22 dataset from sql db and table
conn = sql.connect("wp_deaths_94_to_22.db")
raw_94_to_22 = pd.read_sql("SELECT * FROM wp_deaths_94_to_22", conn)

# Making a working copy
df_94_to_22 = raw_94_to_22.copy()

# Checking the shape
print(f"There are {df_94_to_22.shape[0]} rows and {df_94_to_22.shape[1]} columns.")

# Checking first 2 rows of the data
df_94_to_22.head(2)

There are 133769 rows and 5 columns.


,month_year,day,name,info,link
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer)
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df_94_to_22.tail(2)

,month_year,day,name,info,link
133767,May 2022,31,Dave Smith,", 72, American sound engineer, founder of Sequential.",https://en.wikipedia.org/wiki/Dave_Smith_(engineer)
133768,May 2022,31,Wang Zherong,", 86, Chinese tank designer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Wang_Zherong


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df_94_to_22.sample(5)

,month_year,day,name,info,link
127227,September 2021,26,José Freire Falcão,", 95, Brazilian Roman Catholic cardinal, bishop of Limoeiro do Norte (1967–1971), archbishop of Teresina (1971–1984) and Brasília (1984–2004), COV...",https://en.wikipedia.org/wiki/Jos%C3%A9_Freire_Falc%C3%A3o
53270,January 2012,4,Eve Arnold,", 99, American photographer.",https://en.wikipedia.org/wiki/Eve_Arnold
20827,April 2002,18,Sir Peter Proby,", 90, British landowner, Lord-Lieutenant of Cambridgeshire.",https://en.wikipedia.org/wiki/Peter_Proby
81763,March 2016,12,John Caldwell,", 87, Australian demographer.",https://en.wikipedia.org/wiki/John_Caldwell_(demographer)
57470,September 2012,3,Mahmoud El-Gohary,", 74, Egyptian football player and coach, complications of a stroke.",https://en.wikipedia.org/wiki/Mahmoud_El-Gohary


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,769 rows and 5 columns in the data from January, 1994 through May, 2022.
- The number of references was scraped separately.

### January 1994 through May 2022 Reference Count Data

In [5]:
# Reading the wp_reference_counts_2 dataset from sql db and table
conn = sql.connect("wp_reference_counts_2.db")
raw_reference_counts = pd.read_sql("SELECT * FROM wp_reference_counts_2", conn)

# Making a working copy
df_reference_counts = raw_reference_counts.copy()

# Checking the shape
print(
    f"There are {df_reference_counts.shape[0]} rows and {df_reference_counts.shape[1]} columns."
)

# Checking first 2 rows of the data
df_reference_counts.head(2)

There are 120368 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Lys_Gauty,5
1,https://en.wikipedia.org/wiki/William_Chappell_(dancer),21


<IPython.core.display.Javascript object>

In [6]:
# Checking last 2 rows of the data
df_reference_counts.tail(2)

,link,num_references
120366,https://en.wikipedia.org/wiki/Shirley_Thomas_(USC_professor),6
120367,https://en.wikipedia.org/wiki/James_Doohan,52


<IPython.core.display.Javascript object>

In [7]:
# Checking a sample of the data
df_reference_counts.sample(5)

,link,num_references
105418,https://en.wikipedia.org/wiki/Abdelhamid_ben_Hadouga,2
75573,https://en.wikipedia.org/wiki/Kazi_Nuruzzaman,6
96984,https://en.wikipedia.org/wiki/Nane_Germon,1
63319,https://en.wikipedia.org/wiki/Alain_Resnais,90
119529,https://en.wikipedia.org/wiki/John_Gower_(politician),2


<IPython.core.display.Javascript object>

#### Observations:
- Here, we see that there are ~13,000 fewer rows for the reference data, indicating some pages were not successfully scraped to obtain the number of references for the individual.
- After combining the three dataframes, we can examine those pages and reattempt scraping them, in order to obtain the missing information.

### June 2022 Data

In [8]:
# Reading the wp_deaths_June_2022 dataset from sql db and table
conn = sql.connect("wp_deaths_June_2022.db")
raw_June_2022 = pd.read_sql("SELECT * FROM wp_deaths_June_2022", conn)

# Making a working copy
df_June_2022 = raw_June_2022.copy()

# Checking the shape
print(f"There are {df_June_2022.shape[0]} rows and {df_June_2022.shape[1]} columns.")

# Checking first 2 rows of the data
df_June_2022.head(2)

There are 145 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,June 2022,8,Mladen Frančić,", 67, Croatian football player and manager (Vrbovec, Podravina, Al-Watani Club).",https://en.wikipedia.org/wiki/Mladen_Fran%C4%8Di%C4%87,1
1,June 2022,6,Valery Ryumin,", 82, Russian cosmonaut (Soyuz 25, Soyuz 32, Soyuz 35).",https://en.wikipedia.org/wiki/Valery_Ryumin,2


<IPython.core.display.Javascript object>

#### Observations:
- The June, 2022 data does not follow the same row order as the previous dataframe, which was in order of day of the month. 
- For continuity, before concatinating the two dataframes, we will sort June, 2022 by day.

In [9]:
# Sorting by day
df_June_2022.sort_values(by="day", inplace=True)

# Re-checking first 2 rows of the data
df_June_2022.head(2)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
20,June 2022,1,István Szőke,", 75, Hungarian footballer (Ferencváros, national team), stroke.",https://en.wikipedia.org/wiki/Istv%C3%A1n_Sz%C5%91ke,2


<IPython.core.display.Javascript object>

In [10]:
# Checking last 2 rows of the data
df_June_2022.tail(2)

,month_year,day,name,info,link,num_references
8,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
5,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [11]:
# Checking a sample of the data
df_June_2022.sample(5)

,month_year,day,name,info,link,num_references
125,June 2022,7,Trudy Haynes,", 95, American journalist (WXYZ-TV, KYW-TV).",https://en.wikipedia.org/wiki/Trudy_Haynes,13
9,June 2022,8,Tarhan Erdem,", 89, Turkish politician, deputy (1977–1980) and minister of industry and technology (1977).",https://en.wikipedia.org/wiki/Tarhan_Erdem,8
23,June 2022,1,Deborah McCrary,", 67, American gospel singer (The McCrary Sisters).",https://en.wikipedia.org/wiki/Deborah_McCrary,23
34,June 2022,1,Sir Gerard Brennan,", 94, Australian lawyer and jurist, justice (1981–1995) and chief justice (1995–1998) of the High Court.",https://en.wikipedia.org/wiki/Gerard_Brennan,19
49,June 2022,2,Nobuyuki Idei,", 84, Japanese businessman, liver failure.",https://en.wikipedia.org/wiki/Nobuyuki_Idei,1


<IPython.core.display.Javascript object>

#### Observations:
- Now, we are ready to combine the three dataframes.

<a id='step2'></a>
## Combining Dataframes

### Adding Number of References to 1994 through May 2022 Data

In [12]:
# Adding num_references column to 1994 through May 2022 data
df_combined = pd.merge(df_94_to_22, df_reference_counts, how="left", on="link")

# Checking first 2 rows of the data
df_combined.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

### Adding June 2022 Data

In [13]:
# Adding Juned 2022 data
df_combined = pd.concat([df_combined, df_June_2022], ignore_index=True)

# Making a working copy
df = df_combined.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133914 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [14]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133912,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133913,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [15]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
619,April 1994,5,Marlon Riggs,", 37, American filmmaker, poet, and gay rights activist, AIDS-related complications.",https://en.wikipedia.org/wiki/Marlon_Riggs,22
8623,May 1997,2,Paulo Freire,", 75, Brazilian educator and philosopher, heart attack.",https://en.wikipedia.org/wiki/Paulo_Freire,74
2793,February 1995,26,Barry Dyson,", 52, English football player, heart attack.",https://en.wikipedia.org/wiki/Barry_Dyson,5
25114,February 2004,14,Marco Pantani,", 34, Italian racing cyclist, winner of Tour de France and Giro d'Italia in 1998, acute cocaine poisoning.",https://en.wikipedia.org/wiki/Marco_Pantani,97
71622,September 2014,17,Peter von Bagh,", 71, Finnish film historian.",https://en.wikipedia.org/wiki/Peter_von_Bagh,4


<IPython.core.display.Javascript object>

#### Confirming Correct Number of Resultant Entries

In [16]:
# Confirming correct number of total rows
print(f"There are {df_94_to_22.shape[0] + df_June_2022.shape[0]} expected total rows.")

del df_94_to_22, df_reference_counts, df_June_2022

There are 133914 expected total rows.


<IPython.core.display.Javascript object>

#### Observations:
- We have successfully combined the three dataframes.
- Now, we can check for data types, duplicates, and missing values.

<a id='step3'></a>
## Checking Data Types, Duplicates, and Null Values

### Data types

In [17]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133914 entries, 0 to 133913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133914 non-null  object
 1   day             133914 non-null  object
 2   name            133903 non-null  object
 3   info            133914 non-null  object
 4   link            133914 non-null  object
 5   num_references  120637 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are 6 columns, all of object type.
- `name` and `num_references` both have missing values.
- The data is in a very raw format.  There are columns that have combined information that will need to be extracted.
- For now, we will leave all as object type.

<a id='step3'></a>
### Duplicate Rows

In [18]:
# Checking duplicate rows
df.duplicated().sum()

9

<IPython.core.display.Javascript object>

#### Observations:
- There are 9 duplicate rows that we will drop now.

In [19]:
# Drop duplicate rows
df.drop_duplicates(inplace=True, ignore_index=True)

# Re-check shape
df.shape

(133905, 6)

<IPython.core.display.Javascript object>

<a id='step4'></a>
### Missing Values

In [20]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000
day               0.000
name              0.008
info              0.000
link              0.000
num_references   11.007
dtype: float64

<IPython.core.display.Javascript object>

In [21]:
# Checking number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    120628
1     13266
2        11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The number of rows missing only 1 value appears consistent with our anticipated missing `num_references`.
- There are 11 rows that are each missing 2 values.  Let us take a closer look at these rows.

In [22]:
# Checking the rows that are missing values for 2 columns
missing_2 = df[df.isnull().sum(axis=1) == 2]
missing_2

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,NaN
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,NaN
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,NaN
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,NaN
35097,November 2007,11,None,,https://en.wikipedia.orgNone,NaN
41075,May 2009,18,None,Either killed in a missile attack or shot:\n,https://en.wikipedia.orgNone,NaN
64771,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,NaN
76024,April 2015,29,None,Notable convicted drug traffickers executed by Indonesian firing squad:\n,https://en.wikipedia.orgNone,NaN
105871,August 2019,2,None,"Japanese convicted murderers, executed by hanging.\n",https://en.wikipedia.orgNone,NaN
106617,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can see that multiple rows are missing `name`, but have the name in `info`, so we can extract it later.  
- The missing link itself is not of concern as it serves only as a means by which to retrieve the `num_references` value.
- However, in the event of any future hard-coding, we will need to avoid using the link as the unique identifier for these entries.
- As there is no associated link for the individual, we can safely replace the NaN `num_references` values for rows with extractable names with 0.
- We can proceed with removing the rows that lack an extractable name, as they also lack other information necessary for the analysis.

In [23]:
# List of rows to keep
keep_rows = [18937, 24985, 27458, 34077, 64771, 106617]

# For loop to replace num_references NaNs with 0 for rows with extractable names
for row in keep_rows:
    df.loc[row, "num_references"] = 0

# List of rows to remove
remove_rows = [index for index in missing_2.index if index not in keep_rows]
del missing_2

# Dropping rows
df.drop(remove_rows, inplace=True)
df.reset_index(inplace=True, drop=True)

# Re-checking shape
df.shape

(133900, 6)

<IPython.core.display.Javascript object>

In [24]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133900 entries, 0 to 133899
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  120634 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are now only 6 rows with missing `name`, corresponding to the names we identified in `info`, that we will extract later.
- The remaining missing values are all for `num_references`, so we can proceed to make another attempt at scraping this information.
- Let us check a sample of these rows.

In [25]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
100253,November 2018,12,Yoshito Kajiya,", 80, Japanese politician, member of the House of Councillors (since 2001), heart failure.",https://en.wikipedia.org/wiki/Yoshito_Kajiya,NaN
92639,October 2017,18,Marino Perani,", 77, Italian football manager and player (Bologna, national team).",https://en.wikipedia.org/wiki/Marino_Perani,NaN
66879,January 2014,22,Akkineni Nageswara Rao,", 90, Indian actor (Dadasaheb Phalke Award, , ) and film producer, (1990), cancer.",https://en.wikipedia.org/wiki/Akkineni_Nageswara_Rao,NaN
60248,January 2013,28,Hattie N. Harrison,", 84, American politician, Member of the Maryland House of Delegates (since 1973).",https://en.wikipedia.org/wiki/Hattie_N._Harrison,NaN
133673,May 2022,28,Rifkat Ibragimov,", 71, Uzbek cinematographer (, ).",https://en.wikipedia.org/w/index.php?title=Rifkat_Ibragimov&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links reveals that the pages contain references.  
- Therefore, they either have a variation in the XPath followed for scraping, or Scrapy had an issue with following their links.
- We will export a dataframe of the links to the pages that need to be re-scraped for `num_references`.

In [26]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df = df[df["num_references"].isna()]["link"]
rescrape_df.to_csv("rescrape_df.csv", index=False)

del rescrape_df

<IPython.core.display.Javascript object>

<a id='step5'></a>
## Missing `num_references` Values

### First Re-scrape with Spider ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py)

#### Observations:
- A second iteration of scraping individual pages for number of references reveals variation in the XPath for those pages.
- The new spider, ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py), used an updated XPath, with the identified variation.
- As before, we will import and merge the data, examine rows still missing `num_references` and re-scrape them, adjusting the spider's XPath again, if needed.

In [27]:
# Reading the refs2 dataset from sql db and table
conn = sql.connect("refs2.db")
raw_refs2 = pd.read_sql("SELECT * FROM refs2", conn)

# Making a working copy
df_refs2 = raw_refs2.copy()

# Checking the shape
print(f"There are {df_refs2.shape[0]} rows and {df_refs2.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs2.head(2)

There are 7365 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Charlotte_Benkner,63
1,https://en.wikipedia.org/wiki/Eugene_Record,11


<IPython.core.display.Javascript object>

In [28]:
# Checking last 2 rows of the data
df_refs2.tail(2)

,link,num_references
7363,https://en.wikipedia.org/wiki/Gunnar_Utterberg,3
7364,https://en.wikipedia.org/wiki/Bill_Sudakis,19


<IPython.core.display.Javascript object>

In [29]:
# Checking a sample of the data
df_refs2.sample(5)

,link,num_references
2625,https://en.wikipedia.org/wiki/Bertalan_Bicskei,4
815,https://en.wikipedia.org/wiki/George_Brown_(judge),8
1577,https://en.wikipedia.org/wiki/Ivor_Spencer,1
7111,https://en.wikipedia.org/wiki/Ewald_Kienle,6
2405,https://en.wikipedia.org/wiki/Margaret_Taylor-Burroughs,24


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 7365 of the missing values.

#### Adding Missing References to Dataframe

In [30]:
# Adding new num_references column to data
df = pd.merge(df, df_refs2, how="left", on="link")
del df_refs2

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
29785,January 2006,11,Mabel Sine Wadsworth,", 95, American birth control activist and wonen's health educator.",https://en.wikipedia.org/wiki/Mabel_Sine_Wadsworth,11,NaN
45905,June 2010,9,Joseph Crescent McKinney,", 81, American Roman Catholic auxiliary bishop of Grand Rapids (1968–2001).",https://en.wikipedia.org/wiki/Joseph_Crescent_McKinney,3,NaN
71992,October 2014,8,Jeen van den Berg,", 86, Dutch Olympic speedskater (1956, 1960), winner of the Elfstedentocht (1954).",https://en.wikipedia.org/wiki/Jeen_van_den_Berg,7,NaN
39346,December 2008,20,Dorothy Sarnoff,", 94, American opera singer, actress and self-help consultant.",https://en.wikipedia.org/wiki/Dorothy_Sarnoff,5,NaN
46024,June 2010,19,Alfred Parsons,", 85, Australian diplomat, High Commissioner to the United Kingdom (1983–1987).",https://en.wikipedia.org/wiki/Alfred_Parsons,4,NaN


<IPython.core.display.Javascript object>

In [31]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
55275,April 2012,23,Peter Boothman,", 68-69, Australian guitarist, composer, and educator.",https://en.wikipedia.org/wiki/Peter_Boothman,0,NaN
22533,January 2003,3,Sid Gillman,", 91, American football coach (San Diego Chargers) and member of the Pro Football Hall of Fame.",https://en.wikipedia.org/wiki/Sid_Gillman,13,NaN
84960,September 2016,4,Melvyn Pignon,", 86, English field hockey player.",https://en.wikipedia.org/wiki/Melvyn_Pignon,4,NaN
22254,November 2002,24,Lewis Samuel Feuer,", 89, American sociologist, philosopher, professor and author.",https://en.wikipedia.org/wiki/Lewis_Samuel_Feuer,10,NaN
2348,January 1995,3,Robley C. Williams,", 86, pioneering American biophysicist and virologist.",https://en.wikipedia.org/wiki/Robley_C._Williams,4,NaN


<IPython.core.display.Javascript object>

In [32]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [33]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    5895
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have nearly 6000 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [34]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
76209,May 2015,11,Bob Light,", 88, American college basketball coach (Appalachian State).",https://en.wikipedia.org/wiki/Bob_Light,NaN
83858,July 2016,4,Abbas Kiarostami,", 76, Iranian film director and screenwriter (, , ), gastrointestinal cancer.",https://en.wikipedia.org/wiki/Abbas_Kiarostami,NaN
77368,July 2015,15,Howard Rumsey,", 97, American modern jazz double bassist, bandleader and nightclub owner.",https://en.wikipedia.org/wiki/Howard_Rumsey,NaN
62585,May 2013,28,Fotis Polymeris,", 93, Greek musician.",https://en.wikipedia.org/wiki/Fotis_Polymeris,NaN
85057,September 2016,10,Frank Masley,", 56, American Olympic luger (1980, 1984, 1988), cancer.",https://en.wikipedia.org/wiki/Frank_Masley,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [35]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_2nd = df[df["num_references"].isna()]["link"]
rescrape_df_2nd.to_csv("rescrape_df_2nd.csv", index=False)
del rescrape_df_2nd

<IPython.core.display.Javascript object>

### Second Re-scrape with Spider ["refs3"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs3.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [36]:
# Reading the refs3 dataset from sql db and table
conn = sql.connect("refs3.db")
raw_refs3 = pd.read_sql("SELECT * FROM refs3", conn)

# Making a working copy
df_refs3 = raw_refs3.copy()

# Checking the shape
print(f"There are {df_refs3.shape[0]} rows and {df_refs3.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs3.head(2)

There are 3633 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Grace_Thaxton,63
1,"https://en.wikipedia.org/wiki/Christopher_Prout,_Baron_Kingsland",6


<IPython.core.display.Javascript object>

In [37]:
# Checking last 2 rows of the data
df_refs3.tail(2)

,link,num_references
3631,https://en.wikipedia.org/wiki/Concepci%C3%B3n_Ram%C3%ADrez,8
3632,https://en.wikipedia.org/wiki/Mary_Mahoney_(physician),6


<IPython.core.display.Javascript object>

In [38]:
# Checking a sample of the data
df_refs3.sample(5)

,link,num_references
2804,https://en.wikipedia.org/wiki/Frank_Buxton,5
3543,"https://en.wikipedia.org/wiki/Hugh_Lowther,_8th_Earl_of_Lonsdale",8
14,https://en.wikipedia.org/wiki/Damian_Steele,19
2340,https://en.wikipedia.org/wiki/Bryony_Brind,5
475,https://en.wikipedia.org/wiki/Raymond_Hawkey,12


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 3633 of the missing values.

#### Adding Missing References to Dataframe

In [39]:
# Adding new num_references column to data
df = pd.merge(df, df_refs3, how="left", on="link")
del df_refs3

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
108305,December 2019,7,Denise D'Ascenzo,", 61, American news anchor (WFSB), heart attack.",https://en.wikipedia.org/wiki/Denise_D%27Ascenzo,13,NaN
128336,November 2021,8,Odd F. Lindberg,", 76, Norwegian journalist and seal inspector.",https://en.wikipedia.org/wiki/Odd_F._Lindberg,11,NaN
53664,January 2012,24,Bruce Riutta,", 67, American Olympic ice hockey player, complications from heart surgery.",https://en.wikipedia.org/wiki/Bruce_Riutta,NaN,3
132101,March 2022,17,Artem Datsyshyn,", 43, Ukrainian ballet dancer, missile attack.",https://en.wikipedia.org/wiki/Artem_Datsyshyn,11,NaN
46858,August 2010,23,George T. Smith,", 93, American politician and jurist, Lieutenant Governor of Georgia (1967–1971), Supreme Court of Georgia (1981–1991).",https://en.wikipedia.org/wiki/George_T._Smith,2,NaN


<IPython.core.display.Javascript object>

In [40]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
102018,February 2019,3,John Sinclair,", 79, Australian conservationist.",https://en.wikipedia.org/wiki/John_Sinclair_(environmentalist),6,NaN
25909,July 2004,9,Mike Woodin,", 38, British Principal Speaker of Green Party of England and Wales and Oxford City Councillor.",https://en.wikipedia.org/wiki/Mike_Woodin,3,NaN
127637,October 2021,12,Raúl Baduel,", 66, Venezuelan military officer and politician, minister of defense (2006–2008), complications from COVID-19.",https://en.wikipedia.org/wiki/Ra%C3%BAl_Baduel,23,NaN
46339,July 2010,12,Thomas P. Morahan,", 78, American politician, member of the New York State Assembly (1981–1982); State Senator (1999–2010), leukemia.",https://en.wikipedia.org/wiki/Thomas_P._Morahan,5,NaN
76283,May 2015,15,Matulidi Jusoh,", 57, Malaysian politician, MP for Dungun (2008–2013), diabetes.",https://en.wikipedia.org/wiki/Matulidi_Jusoh,9,NaN


<IPython.core.display.Javascript object>

In [41]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [42]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    2260
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have 2260 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [43]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
89401,April 2017,21,Maria Zhorella Fedorova,", 101, Austrian-born American opera singer and teacher.",https://en.wikipedia.org/wiki/Maria_Zhorella_Fedorova,NaN
107456,October 2019,25,Rudolf G. Wagner,", 77, German sinologist.",https://en.wikipedia.org/wiki/Rudolf_G._Wagner,NaN
92182,September 2017,24,Barbara Blaine,", 61, American founder of SNAP, heart disease.",https://en.wikipedia.org/wiki/Barbara_Blaine,NaN
80548,January 2016,15,Ken Judge,", 58, Australian football player (East Fremantle, Hawthorn, Brisbane Bears) and coach (Hawthorn, West Coast Eagles), cancer.",https://en.wikipedia.org/wiki/Ken_Judge,NaN
91975,September 2017,11,Sir Peter Hall,", 86, British theatre, opera and film director, director of the National Theatre (1973–1988), dementia.",https://en.wikipedia.org/wiki/Peter_Hall_(director),NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [44]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_3rd = df[df["num_references"].isna()]["link"]
rescrape_df_3rd.to_csv("rescrape_df_3rd.csv", index=False)
del rescrape_df_3rd

<IPython.core.display.Javascript object>

### Third Re-scrape with Spider ["refs4"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs4.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [45]:
# Reading the refs4 dataset from sql db and table
conn = sql.connect("refs4.db")
raw_refs4 = pd.read_sql("SELECT * FROM refs4", conn)

# Making a working copy
df_refs4 = raw_refs4.copy()

# Checking the shape
print(f"There are {df_refs4.shape[0]} rows and {df_refs4.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs4.head(2)

There are 1960 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_French_supercentenarians,157
1,https://en.wikipedia.org/wiki/Bob_Kahler,3


<IPython.core.display.Javascript object>

In [46]:
# Checking last 2 rows of the data
df_refs4.tail(2)

,link,num_references
1958,https://en.wikipedia.org/wiki/Jorge_Sampaio,73
1959,https://en.wikipedia.org/wiki/Irma_Kalish,14


<IPython.core.display.Javascript object>

In [47]:
# Checking a sample of the data
df_refs4.sample(5)

,link,num_references
1785,https://en.wikipedia.org/wiki/Ilse_Uyttersprot,2
388,https://en.wikipedia.org/wiki/Myer_Fredman,1
258,https://en.wikipedia.org/wiki/Mirkka_Rekola,2
1570,https://en.wikipedia.org/wiki/Lothar_Zenetti,9
1485,https://en.wikipedia.org/wiki/Daniel_Puckel,2


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 1960 of the missing values.

#### Adding Missing References to Dataframe

In [48]:
# Adding new num_references column to data
df = pd.merge(df, df_refs4, how="left", on="link")
del df_refs4

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
39252,December 2008,13,Shan Lloyd,", 55, British journalist, wife of actor Hugh Lloyd.",https://en.wikipedia.org/wiki/Shan_Lloyd,2,NaN
67688,February 2014,28,Lee Lorch,", 98, American desegregation activist.",https://en.wikipedia.org/wiki/Lee_Lorch,20,NaN
39454,December 2008,31,Stan Kielty,", 83, English rugby league player (Halifax, national team).",https://en.wikipedia.org/wiki/Stan_Kielty,6,NaN
101335,January 2019,6,Gregg Rudloff,", 63, American sound mixer (Oscar, 1990, 2000), 2016 winner (, , ), suicide.",https://en.wikipedia.org/wiki/Gregg_Rudloff,9,NaN
8139,February 1997,27,"Edward J. McCormack, Jr.",", 73, American lawyer and politician, complications from lung cancer.",https://en.wikipedia.org/wiki/Edward_J._McCormack_Jr.,11,NaN


<IPython.core.display.Javascript object>

In [49]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
88653,March 2017,13,Maxx Kidd,", 75, American music producer.",https://en.wikipedia.org/wiki/Maxx_Kidd,11,NaN
98187,July 2018,22,Tony Sparano,", 56, American football coach (Dallas Cowboys, Oakland Raiders, Miami Dolphins), arteriosclerotic heart disease.",https://en.wikipedia.org/wiki/Tony_Sparano,23,NaN
117227,November 2020,5,Jacques Glowinski,", 84, French pharmacist.",https://en.wikipedia.org/wiki/Jacques_Glowinski,19,NaN
15154,January 2000,14,Meche Barba,", 77, American-born Mexican film actress and dancer, heart attack.",https://en.wikipedia.org/wiki/Meche_Barba,1,NaN
42380,September 2009,16,Timothy Bateson,", 83, British actor.",https://en.wikipedia.org/wiki/Timothy_Bateson,5,NaN


<IPython.core.display.Javascript object>

In [50]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [51]:
# Checking remaining missing values
df.isna().sum()

month_year          0
day                 0
name                6
info                0
link                0
num_references    300
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 300 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [52]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
125299,July 2021,19,Mary Ward,", 106, Australian actress (, , ).",https://en.wikipedia.org/wiki/Mary_Ward_(actress),NaN
117051,October 2020,29,Essex Johnson,", 74, American football player (Cincinnati Bengals).",https://en.wikipedia.org/wiki/Essex_Johnson,NaN
119808,January 2021,23,Tom Stevens,", 64, American musician (The Long Ryders).",https://en.wikipedia.org/wiki/Tom_Stevens_(musician),NaN
120395,February 2021,8,Robert Desmond Meikle,", 97, Irish botanist.",https://en.wikipedia.org/wiki/Robert_Desmond_Meikle,NaN
133666,May 2022,28,Edava Basheer,", 78, Indian singer.",https://en.wikipedia.org/w/index.php?title=Edava_Basheer&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [53]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_4th = df[df["num_references"].isna()]["link"]
rescrape_df_4th.to_csv("rescrape_df_4th.csv", index=False)
del rescrape_df_4th

<IPython.core.display.Javascript object>

### Fourth Re-scrape with Spider ["refs5"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs5.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [54]:
# Reading the refs5 dataset from sql db and table
conn = sql.connect("refs5.db")
raw_refs5 = pd.read_sql("SELECT * FROM refs5", conn)

# Making a working copy
df_refs5 = raw_refs5.copy()

# Checking the shape
print(f"There are {df_refs5.shape[0]} rows and {df_refs5.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs5.head(2)

There are 208 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Mohamed_Haytham_Khayat,5
1,https://en.wikipedia.org/wiki/Sid_McCray,6


<IPython.core.display.Javascript object>

In [55]:
# Checking last 2 rows of the data
df_refs5.tail(2)

,link,num_references
206,https://en.wikipedia.org/wiki/Anna_Cataldi,9
207,https://en.wikipedia.org/wiki/Allan_Egolf,4


<IPython.core.display.Javascript object>

In [56]:
# Checking a sample of the data
df_refs5.sample(5)

,link,num_references
91,https://en.wikipedia.org/wiki/Mahaveer_Bhagora,3
159,https://en.wikipedia.org/wiki/William_Clutz,24
170,https://en.wikipedia.org/wiki/Alfredo_Petit-Vergel,0
176,https://en.wikipedia.org/wiki/Olivier_Maire,6
160,https://en.wikipedia.org/wiki/Alain_Barrau,6


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 208 of the missing values.

#### Adding Missing References to Dataframe

In [57]:
# Adding new num_references column to data
df = pd.merge(df, df_refs5, how="left", on="link")
del df_refs5

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
43341,December 2009,6,Daouda Sow,", 76, Senegalese politician, President of National Assembly (1984–1998).",https://en.wikipedia.org/wiki/Daouda_Sow_(politician),3,NaN
20173,January 2002,18,Celso Daniel,", 50, Brazilian politician and mayor, murdered.",https://en.wikipedia.org/wiki/Celso_Daniel,7,NaN
126794,September 2021,10,Michael Chapman,", 80, English singer-songwriter and guitarist ().",https://en.wikipedia.org/wiki/Michael_Chapman_(singer),6,NaN
40428,March 2009,20,Vicente Gandía,", 84, Mexican artist of Spanish origin, heart attack.",https://en.wikipedia.org/wiki/Vicente_Gand%C3%ADa,10,NaN
45955,June 2010,13,Nelson Wallulatum,", 84, American Wasco tribe leader, chief of the Wasco Indians (since 1959), founder of The Museum at Warm Springs.",https://en.wikipedia.org/wiki/Nelson_Wallulatum,2,NaN


<IPython.core.display.Javascript object>

In [58]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
18626,June 2001,30,Joe Henderson,", 64, American jazz tenor saxophonist.",https://en.wikipedia.org/wiki/Joe_Henderson,6,NaN
7143,October 1996,24,Gladwyn Jebb,", 96, British civil servant, diplomat and politician.",https://en.wikipedia.org/wiki/Gladwyn_Jebb,7,NaN
100790,December 2018,12,Ferenc Kósa,", 81, Hungarian film director (, ).",https://en.wikipedia.org/wiki/Ferenc_K%C3%B3sa,2,NaN
3434,May 1995,15,Pia Tassinari,", 91, Italian opera singer who was first a soprano and later a mezzo-soprano.",https://en.wikipedia.org/wiki/Pia_Tassinari,2,NaN
3943,July 1995,24,Jerzy Toeplitz,", 85, Russian film maker.",https://en.wikipedia.org/wiki/Jerzy_Toeplitz,5,NaN


<IPython.core.display.Javascript object>

In [59]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [60]:
# Checking remaining missing values
df.isna().sum()

month_year         0
day                0
name               6
info               0
link               0
num_references    92
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 92 remaining missing values for `num_references`.
- Let us examine the remaining rows with missing values more closely.

In [61]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()]

,month_year,day,name,info,link,num_references
12476,October 1998,29,Sjoerdtsje Faber,", 83, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sjoerdje_Faber&action=edit&redlink=1,NaN
15859,May 2000,10,Bill Foster,", 68, American entertainer.",https://en.wikipedia.org/w/index.php?title=Bill_Foster_(performer)&action=edit&redlink=1,NaN
18561,June 2001,20,Demetreus Nix,"Douglas Scott, 20, High-school student murdered by .",https://en.wikipedia.org/w/index.php?title=Demetreus_Nix&action=edit&redlink=1,NaN
19905,December 2001,18,Sietske Pasveer,", 86, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sietske_Pasveer&action=edit&redlink=1,NaN
22942,March 2003,3,Gilbert Wheeler Beebe,", 90, American epidemiologist and statistician, conducted ground-breaking radiation exposure studies.",https://en.wikipedia.org/w/index.php?title=Gilbert_Wheeler_Beebe&action=edit&redlink=1,NaN
23291,April 2003,27,Charles A. Marvin,", 73, American district attorney and judge.",https://en.wikipedia.org/w/index.php?title=Charles_A._Marvin&action=edit&redlink=1,NaN
24346,October 2003,15,Ray Kuhlman,", 84, American pilot and businessman.",https://en.wikipedia.org/w/index.php?title=Ray_Kuhlman&action=edit&redlink=1,NaN
33653,May 2007,20,Moses Siregar,", 96, Maritime Chef and American World War 2 Veteran",https://en.wikipedia.org/w/index.php?title=Moses_Siregar&action=edit&redlink=1,NaN
35963,January 2008,31,Michael A. Dions,", 90, American Olympic swimmer, natural causes",https://en.wikipedia.org/w/index.php?title=Michael_A._Dions&action=edit&redlink=1,NaN
36198,February 2008,19,Samuel Champkin,", 28, American singer in Metal band Tech Giants, car crash.",https://en.wikipedia.org/w/index.php?title=Samuel_Champkin&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- None of the remaining links direct to a personal page for the individual, as such a page does not yet exist, at the time of this scraping. 
- As such, as the rows contain the other elements necessary for analysis, it is safe to replace the missing `num_references` values with 0, for these rows.

In [62]:
# Fill remaining missing values for num_references with 0
df["num_references"].fillna(0, inplace=True)

# Recheck missing values
df.isna().sum()

month_year        0
day               0
name              6
info              0
link              0
num_references    0
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- All of the missing values for `num_references` have been addressed.
- The 6 remaining missing values for `name` will be fixed during data cleaning.
- We now have our complete raw dataset, which we will also export to a SQLite database.
- Then it is time to start cleaning the data.

### Exporting Dataset to SQLite Database [wp_life_expect_raw_complete.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_raw_complete.db)

In [63]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_life_expect_raw_complete.db")
df.to_sql("wp_life_expect_raw_complete", conn, index=False)

133900

<IPython.core.display.Javascript object>

# [Proceed to Data Cleaning Part 1](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean1_thanak_2022_06_13.ipynb)

[Return to README](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/README.md#explore-the-project)